In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from lxml import etree
from datetime import datetime
from random import randint
from time import sleep
import random
import urllib.parse

In [2]:
user_agent_list = [
    # Chrome
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (Windows NT 5.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.116 Safari/537.36'
    # Firefox
    'Mozilla/4.0 (compatible; MSIE 9.0; Windows NT 6.1)',
    'Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.1; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (Windows NT 6.2; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.0; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.3; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; WOW64; Trident/6.0)',
    'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; Trident/6.0)',
    'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0; .NET CLR 2.0.50727; .NET CLR 3.0.4506.2152; .NET CLR 3.5.30729)',
    'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:59.0) Gecko/20100101 Firefox/59.0'
]

def get_random_agent():
    return random.choice(user_agent_list)

# https://scrapeops.io/app/dashboard
# backup:18141225-2ab9-40ab-8912-e268ac5b8143
API_KEY = '607e459a-3447-4295-bd47-5fc7607dae7a'

def get_scrapeops_url(url):
    payload = {'api_key': API_KEY, 'url': url, 'bypass': 'cloudflare'}
    proxy_url = 'https://proxy.scrapeops.io/v1/?' + urllib.parse.urlencode(payload)
    return proxy_url

In [3]:
page_num = 5

def get_post(page, user_agent):
    URL = "https://www.cw.com.tw/subchannel.action?idSubChannel=9&page={}".format(page)
    
    HEADERS = {
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'user-agent': user_agent,
        'access-control-allow-credentials': 'true'
        }

    webpage = requests.get(get_scrapeops_url(URL), headers=HEADERS)
    soup = BeautifulSoup(webpage.content, "html.parser")
    dom = etree.HTML(str(soup))
    
    date_list = [datetime.strptime(str(s).replace('\n', '').replace('\u3000', '').replace('\xa0', '').replace(' ', ''), '%Y-%m-%d').date() for s in dom.xpath("//section[@class='article']//div[@class='caption']//time/text()")]
    title_list = [str(s).replace('\n', '').replace('\u3000', '').replace('\xa0', '') for s in dom.xpath("//section[@class='article']//div[@class='caption']//h3/a/text()")]
    text_list = [str(s) for s in dom.xpath("//section[@class='article']//div[@class='caption']//p/text()")]
    link_list = [str(s) for s in dom.xpath("//section[@class='article']//div[@class='caption']//h3/a/@href")]
    df = pd.DataFrame(
    {
        'date': date_list,
        'title': title_list,
        'text': text_list,
        'link':link_list
    })
    return df

In [4]:
df = pd.DataFrame(columns=['date', 'title', 'text', 'link'])

for i in range(page_num):
    df = df.append(get_post(i, get_random_agent()), ignore_index=True)
    sleep(randint(3,15))

/var/folders/6f/v4g3dkkj0b54w0sc2l43bkqm0000gn/T/ipykernel_37864/49057815.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(get_post(i, get_random_agent()), ignore_index=True)


In [5]:
df.to_csv('cw.csv', encoding='utf-8-sig', columns=['date', 'title', 'text', 'link'])
tmp = pd.read_csv('cw.csv', index_col=0)
tmp

,date,title,text,link
0,2022-11-11,鴻海劉揚偉首度說明鄭州廠內情競爭者要搶單？「機會不大！」,鴻海11月10日召開法說會，董事長劉揚偉面對富士康鄭州廠疫情後續，回答法人媒體質疑。他承認第...,https://www.cw.com.tw/article/5123506
1,2022-11-09,「人才基地計畫」串連產官學研，打造半導體實務戰力人才,隨著5G、物聯網、人工智慧(AI)等創新科技引領臺灣科技發展，對於科技人才的需求也逐年增加，...,https://www.cw.com.tw/article/5123397
2,2022-11-04,iKala Cloud攜手Alkira助企業網路雲端化，全面提升企業營運效能,大缺工時代，企業雲端轉型，最缺雲端人才。iKala旗下雲端事業iKala Cloud與雲端網...,https://www.cw.com.tw/article/5123408
3,2022-11-01,歐洲直擊｜藏身小城的台積大腦、EUV誕生地：少了台灣，半導體一夕垮掉,全球最強大的半導體實驗室，藏在人口比台東市還少的比利時小城。一家非營利研究機構，卻是台積電、...,https://www.cw.com.tw/article/5123405
4,2022-11-01,老文件揭張忠謀40年前神預言晶圓教父的產地，要感謝一位教授,【陳良榕專欄】最先提出晶圓代工的，究竟是曹興誠還是張忠謀？新挖掘的老文件，揭露張忠謀40年前...,https://www.cw.com.tw/article/5123368
5,2022-10-31,科技時代下的文化產業革新 — 科技結合文化，釋放產業想像,由文化內容策進院主辦的TCCF創意內容大會-未來內容展，今年主題為「擴增宇宙 運算中」（Fu...,https://www.cw.com.tw/article/5123328
6,2022-10-29,為什麼大家搶挖比特幣？一次搞懂Web 3.0加密貨幣的數位資產價值,人人自己「印鈔票」的概念，其實已經在發生。,https://www.cw.com.tw/article/5123340
7,2022-10-28,440億美元收購》馬斯克變Twitter老闆會發生什麼事？,馬斯克終於在週四晚間以440億美元，完成了買下Twitter的交易。一上任就瞬間開除4名高階...,https://www.cw.com.tw/article/5123332
8,2022-10-27,疫情大餅吃完了？矽谷科技巨頭將迎來寒冬,過去十年，科技業帶領著美國的經濟發展，並在Covid-19肆虐時提振了股市，但如今面臨多重不...,https://www.cw.com.tw/article/5123313
9,2022-10-26,台積本益比接近金融海嘯時期高盛直指恐面臨六大風險,兩岸地緣政治風險嚇壞全球投資人，衝擊台積電股價25日下探兩年多來新低。外資高盛證券認為，台積...,https://www.cw.com.tw/article/5123289
